# Spatial Prediction

<hr>

**How to Model Covariance Matrices?**<br>
Often, we will be in a situation where we have made some observations of a certain location, but do not have the covariances with another location.

One can assume that the covariance between a pair of cities can be determined by its distance (spatial correlation). In this case,

$\displaystyle  \textsf{Cov}(X_1,X_2) = k(Z_1,Z_2)$

where $k(Z_1, Z_2)$ is some covariance function, for example, RBF kernel (squared exponential), which decays exponentially

$\displaystyle  \textsf{Cov}(X_1,X_2) = k(Z_1, Z_2) = \exp (- \frac{\| Z_1 - Z_2\| ^2}{2\ell ^2})$

where $\ell$ is a parameter to be estimated, called the **length-scale**. Covariance functions often go by the name **kernels** ($k$) as well, which are a broader class of functions.

$
\displaystyle  \mathbf{\Sigma_N} = \begin{bmatrix}  \mathbf{k(x_1, x_1)} &  \mathbf{k(x_1, x_2)}\\ \mathbf{k(x_2, x_1)} &  \mathbf{k(x_2, x_2)} \end{bmatrix}
$

which is also called a kernel ($k$) matrix, $K_N$


High-level idea: 
- Determine a probabilistic model, a multivariate Gaussian with a prior covariance which is decaying with distance
- Observe values $y_1, \dots, y_N$ for some locations, $x_1, \dots, x_N$
- Obtain a posterior distribution for the unobserved variables $Y_*$: $p(Y_* | y_1, \dots, y_N)$ Gaussian

Interpolation with Gaussian processes ([Kriging](https://en.wikipedia.org/wiki/Kriging))
- Build a kernel matrix $K_N$ of $N$ observations
- For a new point $x_*$: compute a vector $k_*^T = [k(x_*, x_1), \dots, k(x_*, x_N)]$

    $\mu_{*|1:N} = \mu_{*} + k_{*}^T K_{N}^{-1} (y_{1:N} - \mu_{1:N})$
    
    $\sigma_{*|1:N}^2 = \sigma_*^2 - k_{*}^T K_{N}^{-1} k_*$

<img alt="Kriging" src="assets/kriging.jpg" width="300">

<img alt="Kriging with temperature sensors" src="assets/kriging_temperatures.jpg" width="600">

<hr>

**Kernel functions**


<hr>

# Basic code
A `minimal, reproducible example`